In [1]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import pandas_datareader.data as web
from numpy import nan as NA

pandas是本书后续内容的首选库。它含有使数据分析工作变得更快更简单的高级数据结构和操作工具。pandas是基于NumPy构建的，让以NumPy为中心的应用变得更加简单。<br>
在过去的4年中，pandas逐渐成长为一个非常大的库，它所能解决的数据处理问题已经比我期望的要多得多了。但随着其范围的扩大，它也逐渐背离了我最初所期望的简洁性和易用性。我希望你在读完本书后，也能像我一样认为它是一个不可或缺的工具。

# pandas的数据结构介绍

要使用pandas，你首先就得熟悉它的两个主要数据结构：Series和DataFrame。虽然它们并不能解决所有问题，但它们为大多数应用提供了一种可靠的、易于使用的基础。

## Series

Series是一种类似于以为数组的对象，它由一组数据（各种NumPy数据类型）以及一组与之相关的数据标签（即索引）组成。仅由一组数据即可产生最简单的Series。

In [2]:
obj = Series([4,7,-5,3])
obj

0    4
1    7
2   -5
3    3
dtype: int64

Series的字符串表现形式为：索引在左边，值在右边。没有指定索引时，默认从0开始的数字指引。

In [3]:
obj.values

array([ 4,  7, -5,  3], dtype=int64)

In [4]:
obj.index

RangeIndex(start=0, stop=4, step=1)

通常，我们希望所创建的Series带有一个可以对各个数据点进行标记的索引：

In [5]:
obj2 = Series([4,7,-5,3], index=['a', 'b', 'c', 'd'])
obj2

a    4
b    7
c   -5
d    3
dtype: int64

In [6]:
obj2.index

Index(['a', 'b', 'c', 'd'], dtype='object')

与普通NumPy数组相比，你可以通过索引的方式选取Series中的单个或一组值：

In [7]:
obj2['a']

4

In [9]:
obj2['d'] = 6

In [10]:
obj2[['c', 'a', 'd']]

c   -5
a    4
d    6
dtype: int64

NumPy数组运算（如根据布尔型数组进行过滤、标量乘法、应用数学函数等）都会保留索引和值之间的链接：

In [11]:
obj2

a    4
b    7
c   -5
d    6
dtype: int64

In [12]:
obj2[obj2 > 0]

a    4
b    7
d    6
dtype: int64

In [13]:
obj2 * 2

a     8
b    14
c   -10
d    12
dtype: int64

In [14]:
np.exp(obj2)

a      54.598150
b    1096.633158
c       0.006738
d     403.428793
dtype: float64

还可以将Series看成是一个定长的有序字典，因为它是索引值到数据值的一个映射。它可以用在许多原本需要字典参数的函数中：

In [15]:
'b' in obj2

True

In [16]:
'e' in obj2

False

如果数据被存放在一个Python字典中，也可以直接通过这个字典来创建Series：

In [17]:
sdata = {
    'Ohio':35000,
    'Texas':71000,
    'Oregon':16000,
    'Utah':5000
}
obj3 = Series(sdata)
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

如果只传入一个字典，则结果Series中的索引就是原字典的键

In [19]:
states = ['Califonia', 'Ohio', 'Oregon', 'Texas']
obj4 = Series(sdata, index=states)
obj4

Califonia        NaN
Ohio         35000.0
Oregon       16000.0
Texas        71000.0
dtype: float64

在这个例子中，sdata中跟states索引相匹配的那3个值会被找出来并放到相应的位置上，但由于California所对应的sdata值找不到，所以其结果就为NaN（即非数字Not a number，在pandas中，它用于表示缺失或NA值）。我将使用缺失（missing）或NA表示缺失数据。pandas的isnull或notnull函数可用于检测缺失数据：

In [21]:
pd.isnull(obj4)

Califonia     True
Ohio         False
Oregon       False
Texas        False
dtype: bool

In [22]:
pd.notnull(obj4)

Califonia    False
Ohio          True
Oregon        True
Texas         True
dtype: bool

Series也有类似的实例方法：

In [23]:
obj4.isnull()

Califonia     True
Ohio         False
Oregon       False
Texas        False
dtype: bool

对于许多应用而言，Series最重要的一个功能是：它在算术运算中会自动对齐不同索引的数据。

In [24]:
obj3

Ohio      35000
Texas     71000
Oregon    16000
Utah       5000
dtype: int64

In [25]:
obj4

Califonia        NaN
Ohio         35000.0
Oregon       16000.0
Texas        71000.0
dtype: float64

In [26]:
obj3 + obj4

Califonia         NaN
Ohio          70000.0
Oregon        32000.0
Texas        142000.0
Utah              NaN
dtype: float64

Series对象本身及其索引都有一个name属性，该属性跟pandas其他的关键功能关系非常密切：

In [27]:
obj4.name = 'population'
obj4.index.name = 'state'
obj4

state
Califonia        NaN
Ohio         35000.0
Oregon       16000.0
Texas        71000.0
Name: population, dtype: float64

Series的索引可以通过赋值的方式就地修改：

In [28]:
obj.index = ['Bob', 'Steve', 'Jeff', 'Ryan']
obj

Bob      4
Steve    7
Jeff    -5
Ryan     3
dtype: int64

## DataFrame

DataFrame是一个表格型的数据结构，它含有一组有序的列，每列可以是不同的值类型（数值，字符串，布尔值）。DataFrame既有行索引也有列索引，他可以被看作有Series组成的字典（共用同一个索引）。跟其他类似的数据结构相比，DataFrame中面向行和面向列的操作基本上是平衡的。其实，DataFrame中的数据是以一个或多个二维块存放的（而不是列表、字典或别的一维数据结构）。<br>
虽然DataFrame是以二维结构保存数据的，但你仍然可以轻松地将其表示为更高维度的数据（层次化索引的表格型结构，这是pandas中许多高级数据处理功能的关键要素）。

构建DataFrame的方法有很多，最常用的一种是直接传入一个由等长或NumPy数组组成的字典：

In [29]:
data = {
    'state':['Ohio','Ohio','Ohio','Nevada','Nevada'],
    'year':[2000,2001,2002,2001,2002],
    'pop':[1.5,1.7,3.6,2.4,2.9]
}
frame = DataFrame(data)
frame

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,1.7
2,Ohio,2002,3.6
3,Nevada,2001,2.4
4,Nevada,2002,2.9


结构DataFrame会自动加上索引（跟Series一样），且全部列会被有序排列：

In [30]:
DataFrame(data, columns=['year','state','pop'])

,year,state,pop
0,2000,Ohio,1.5
1,2001,Ohio,1.7
2,2002,Ohio,3.6
3,2001,Nevada,2.4
4,2002,Nevada,2.9


跟Series一样，如果传入的列在数据中找不到，就会产生NA值：

In [31]:
frame2 = DataFrame(data, columns=['year','state','pop','debt'], index=['one','two','three','four','five'])
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,NaN
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,NaN
five,2002,Nevada,2.9,NaN


In [32]:
frame2.columns

Index(['year', 'state', 'pop', 'debt'], dtype='object')

通过类似字典标记的方式或属性的方式，可以将DataFrame的列获取为一个Series：

In [33]:
frame2['state']

one        Ohio
two        Ohio
three      Ohio
four     Nevada
five     Nevada
Name: state, dtype: object

In [34]:
frame2.year

one      2000
two      2001
three    2002
four     2001
five     2002
Name: year, dtype: int64

注意，返回的Series拥有原DataFrame相同的索引，且其name属性也已经被相应地设置好了。行也可以通过位置或名称的方式进行获取，比如用索引字段loc：

In [35]:
frame2.loc['three']

year     2002
state    Ohio
pop       3.6
debt      NaN
Name: three, dtype: object

列可以通过赋值的方式进行修改。例如，我们可以给那个空的debt列附上一个量值或一组值：

In [36]:
frame2['debt'] = 16.5
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,16.5
two,2001,Ohio,1.7,16.5
three,2002,Ohio,3.6,16.5
four,2001,Nevada,2.4,16.5
five,2002,Nevada,2.9,16.5


In [37]:
frame2['debt'] = np.arange(5)
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,0
two,2001,Ohio,1.7,1
three,2002,Ohio,3.6,2
four,2001,Nevada,2.4,3
five,2002,Nevada,2.9,4


将列表或数组赋值给某个列时，其长度必须跟DataFrame的长度相匹配。如果赋值的是一个Series，就会精确匹配DataFrame的索引，所有的空位都将被填上缺失值：

In [38]:
val = Series([-1.2, -1.5, -1.7], index=['two', 'four', 'five'])
frame2['debt'] = val
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


为不存在的列赋值会创建出一个新列。关键字del用于删除列

In [39]:
frame2['eastern'] = frame2.state == 'Ohio'
frame2

,year,state,pop,debt,eastern
one,2000,Ohio,1.5,NaN,True
two,2001,Ohio,1.7,-1.2,True
three,2002,Ohio,3.6,NaN,True
four,2001,Nevada,2.4,-1.5,False
five,2002,Nevada,2.9,-1.7,False


In [40]:
del frame2['eastern']
frame2

,year,state,pop,debt
one,2000,Ohio,1.5,NaN
two,2001,Ohio,1.7,-1.2
three,2002,Ohio,3.6,NaN
four,2001,Nevada,2.4,-1.5
five,2002,Nevada,2.9,-1.7


通过索引方式返回的列只是相应数据的视图而已，并不是副本。因此，对返回的Series所做的任何就地修改都会反映到源DataFrame上。通过Series的copy方法即可显式地复制列。

另一种常见的数据形式是嵌套字典（也就是字典的字典）：

In [41]:
pop = {'Nevada': {2001:2.4, 2002:2.9}, 'Ohio': {2000:1.5, 2001:1.7, 2002:3.6}}

如果将它传给DataFrame，它就会被解释为：外层字典的键作为列，内层键则作为行索引：

In [42]:
frame3 = DataFrame(pop)
frame3

,Nevada,Ohio
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


当然，你也可以对该结果进行转置

In [43]:
frame3.T

,2000,2001,2002
Nevada,NaN,2.4,2.9
Ohio,1.5,1.7,3.6


内层字典的键会被合并、排序以形成最终的索引。如果显式指定了索引，则不会这样：

In [44]:
DataFrame(pop, index=[2001, 2002, 2003])

,Nevada,Ohio
2001,2.4,1.7
2002,2.9,3.6
2003,NaN,NaN


由Series组成的字典差不多也是一样的用法：

In [46]:
pdata = {'Ohio': frame3['Ohio'][:-1], 'Nevada': frame3['Nevada'][:2]}
DataFrame(pdata)

,Ohio,Nevada
2000,1.5,NaN
2001,1.7,2.4


如果设置了DataFrame的index和columns的names属性，则这些信息也会被显示出来：

In [47]:
frame3.index.name = 'year'
frame3.columns.name = 'state'

In [48]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


跟Series一样，values属性也会以二维ndarray的形式返回DataFrame中的数据：

In [49]:
frame3.values

array([[nan, 1.5],
       [2.4, 1.7],
       [2.9, 3.6]])

如果DataFrame各列的数据类型不同，则值数组的数据类型就会选用能兼容所有列的数据类型：

In [50]:
frame2.values

array([[2000, 'Ohio', 1.5, nan],
       [2001, 'Ohio', 1.7, -1.2],
       [2002, 'Ohio', 3.6, nan],
       [2001, 'Nevada', 2.4, -1.5],
       [2002, 'Nevada', 2.9, -1.7]], dtype=object)

## 索引对象

pandas的索引对象负责管理轴标签和其他元数据（比如轴名称等）。构建Series或DataFrame时，所用到的任何数组或其他序列的标签都会被转换成一个index：

In [51]:
obj = Series(range(3), index=['a', 'b', 'c'])
index = obj.index
index

Index(['a', 'b', 'c'], dtype='object')

In [52]:
index[1:]

Index(['b', 'c'], dtype='object')

index对象是不可以修改的（immutable），这样才能使index对象在多个数据结构之间安全共享

In [53]:
index[1] = 'd'

TypeError: Index does not support mutable operations

In [54]:
index = pd.Index(np.arange(3))
obj2 = Series([1.5,2.5,0], index=index)
obj2

0    1.5
1    2.5
2    0.0
dtype: float64

In [55]:
obj2.index is index

True

除了长得像数组，Index的功能也类似一个固定大小的集合：

In [56]:
frame3

state,Nevada,Ohio
year,,
2000,NaN,1.5
2001,2.4,1.7
2002,2.9,3.6


In [57]:
'Ohio' in frame3.columns

True

In [58]:
2003 in frame3.index

False

# 基本功能

本节中，我将介绍操作Series和DataFrame中的数据的基本手段。后续章节将更加深入的挖掘pandas在数据分析和处理方面的功能。本书不是pandas库的详尽文档，主要关注的是最重要的功能，哪些不大常用的内容就交给你自己去摸索吧。

## 重新索引

pandas对象的一个重要方法是reindex，其作用是创建一个适应新索引的新对象。以之前的一个简单实例来说：

In [59]:
obj = Series([4.5,7.2,-5.3,3.6], index=['d','b','a','c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

调用该Series的reindex将会根据新的索引进行重排。如果某个索引值当前不存在，就引入缺失值：

In [60]:
obj2 = obj.reindex(['a','b','c','d','e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

In [65]:
obj.reindex(['a', 'b', 'c', 'd', 'e'], fill_value=0)

a   -5.3
b    7.2
c    3.6
d    4.5
e    0.0
dtype: float64

对于时间序列这样的有序数据，重新索引时可能需要做一些插值处理。method选项即可达到此目的，例如，使用ffill可以实现前向填充：

In [66]:
obj3 = Series(['blue','purple','yellow'], index=[0,2,4])
obj3.reindex(range(6), method='ffill')

0      blue
1      blue
2    purple
3    purple
4    yellow
5    yellow
dtype: object

对于DataFrame，reindex可以修改索引，如果仅传入一个序列，则会重新索引行：

In [67]:
frame = DataFrame(np.arange(9).reshape((3,3)), index=['a','b','c'], columns=['Ohio','Texas','California'])
frame

,Ohio,Texas,California
a,0,1,2
b,3,4,5
c,6,7,8


In [68]:
frame2 = frame.reindex(['a','b','c','d'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,3.0,4.0,5.0
c,6.0,7.0,8.0
d,NaN,NaN,NaN


使用columns关键字即可重新索引列：

In [69]:
states = ['Texas','Utah','California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
b,4,NaN,5
c,7,NaN,8


也可以同时对行和列进行重新索引，而插入则只能按行应用

In [70]:
frame.reindex(index=['a','b','c','d'], columns=states).ffill()

,Texas,Utah,California
a,1.0,NaN,2.0
b,4.0,NaN,5.0
c,7.0,NaN,8.0
d,7.0,NaN,8.0


利用loc的标签索引功能，重新索引任务可以变得更简洁：

In [73]:
frame.loc[['a', 'b', 'c', 'd'], states]

D:\software\anaconda\install\lib\site-packages\pandas\core\indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,Texas,Utah,California
a,1.0,NaN,2.0
b,4.0,NaN,5.0
c,7.0,NaN,8.0
d,NaN,NaN,NaN


## 丢弃指定轴上的项

丢弃某条轴上的一个或多个项很简单，只要有一个索引数组或列表即可。由于需要执行一些数据整理和集合逻辑，所以drop方法返回的是一个在指定轴上删除了指定值的新对象。

In [74]:
obj = Series(np.arange(5), index=['a','b','c','d','e'])
new_obj = obj.drop('c')
new_obj

a    0
b    1
d    3
e    4
dtype: int32

对于DataFrame，可以删除任意轴上的索引值：

In [75]:
data = DataFrame(np.arange(16).reshape((4,4)), index=['Ohio','Colorado','Utah','New York'], columns=['one','two','three','four'])
data.drop(['Colorado', 'Ohio'])

,one,two,three,four
Utah,8,9,10,11
New York,12,13,14,15


In [76]:
data.drop('two', axis=1)

,one,three,four
Ohio,0,2,3
Colorado,4,6,7
Utah,8,10,11
New York,12,14,15


In [77]:
data.drop(['two', 'four'], axis=1)

,one,three
Ohio,0,2
Colorado,4,6
Utah,8,10
New York,12,14


## 索引、选取和过滤

Series索引（obj[...]）的工作方式类似于NumPy数组的索引，只不过Series的索引值不只是整数。

In [78]:
obj = Series(np.arange(4), index=['a','b','c','d'])
obj

a    0
b    1
c    2
d    3
dtype: int32

In [79]:
obj['b']

1

In [80]:
obj[1]

1

In [81]:
obj[[1,3]]

b    1
d    3
dtype: int32

In [82]:
obj[2:4]

c    2
d    3
dtype: int32

In [83]:
obj[['b', 'a', 'd']]

b    1
a    0
d    3
dtype: int32

In [84]:
obj[obj < 2]

a    0
b    1
dtype: int32

利用标签的切片运算与普通的Python切片运算不同，是闭区间

In [85]:
obj['b':'c']

b    1
c    2
dtype: int32

In [86]:
obj['b','c'] = 5
obj

a    0
b    5
c    5
d    3
dtype: int32

In [87]:
data = DataFrame(np.arange(16).reshape((4,4)), index=['Ohio','Colorado','Utah','New York'], columns=['one','two','three','four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [88]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int32

In [89]:
data[['three','one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


这种索引方式有几个特殊的情况。首先通过切片或布尔型数组选取行：

In [90]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [91]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


有些读者可能会认为这不太合乎逻辑，但这种语法的的确确来源于实践。另一种用法是通过布尔型DataFrame（比如下面这个由标量比较运算得出的）进行索引：

In [92]:
data < 5

,one,two,three,four
Ohio,True,True,True,True
Colorado,True,False,False,False
Utah,False,False,False,False
New York,False,False,False,False


In [93]:
data[data < 5] = 0
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


这段代码的目的是使DataFrame在语法上更像ndarray。

为了在DataFrame的行上进行标签索引，我引入了专门的索引资料loc。它使你可以通过NumPy式的标记法以及轴标签从DataFrame中选取行和列的字节。

In [101]:
data

,one,two,three,four
Ohio,0,0,0,0
Colorado,0,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [94]:
data.loc['Colorado',['two','three']]

two      5
three    6
Name: Colorado, dtype: int32

In [100]:
data.iloc[2, [3, 0, 1]]

four    11
one      8
two      9
Name: Utah, dtype: int32

## 算术运算和数据对齐

pandas最重要的一个功能是，它可以对不同索引的对象进行算术运算。在将对象相加时，如果存在不同的索引对，则结果的索引就是该索引对的并集。

In [102]:
s1 = Series([7.3,-2.5,3.4,1.5], index=['a','c','d','e'])
s2 = Series([-2.1,3.6,-1.5,4,3.1], index=['a','c','e','f','g'])
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

自动的数据对齐操作在不重叠的索引处引入了NA值。缺失值会在算术运算过程中传播。

对于DataFrame，对齐操作会同时发生在行和列上：

In [103]:
df1 = DataFrame(np.arange(9).reshape((3,3)), index=['Ohio','Texas','Colorado'], columns=list('bcd'))
df2 = DataFrame(np.arange(12).reshape((4,3)), index=['Utah','Ohio','Texas','Oregon'], columns=list('bde'))
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


## 在算术方法中填充值

在对不同索引的对象进行算术运算时，你可能希望当一个对象中某个轴标签在另一个对象中找不到时填充一个特殊值（比如0）：

In [104]:
df1 = DataFrame(np.arange(12).reshape((3,4)), columns=list('abcd'))
df2 = DataFrame(np.arange(20).reshape((4,5)), columns=list('abcde'))

In [105]:
df1

,a,b,c,d
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11


In [106]:
df2

,a,b,c,d,e
0,0,1,2,3,4
1,5,6,7,8,9
2,10,11,12,13,14
3,15,16,17,18,19


In [107]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,11.0,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


In [108]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,11.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


与此类似，在对Series或DataFrame重新索引时，也可以指定一个填充值：

In [109]:
df1.reindex(columns=df2.columns, fill_value=0)

,a,b,c,d,e
0,0,1,2,3,0
1,4,5,6,7,0
2,8,9,10,11,0


## DataFrame和Series之间的运算

跟NumPy数组一样，DataFrame和Series直接算术运算也是有明确规定的。先来看一个具有启发性的例子，计算一个二维数组与其某行之间的差

In [2]:
arr = np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [3]:
arr[0]

array([0, 1, 2, 3])

In [4]:
arr - arr[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

这就叫做广播（broadcasting），后面章节会详细讲解。

In [5]:
frame = DataFrame(np.arange(12).reshape((4,3)), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [6]:
series = frame.iloc[0]
series

b    0
d    1
e    2
Name: Utah, dtype: int32

默认情况下，DataFrame和Series之间的算术运算会将Series的索引匹配到DataFrame的列，然后沿着行一直向下广播：

In [7]:
frame - series

,b,d,e
Utah,0,0,0
Ohio,3,3,3
Texas,6,6,6
Oregon,9,9,9


如果某个索引值在DataFrame的列或Series的索引中找不到，则参与运算的两个对象就会被重新索引以形成并集：

In [8]:
series2 = Series(range(3), index=['b', 'e', 'f'])

In [9]:
frame + series2

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


如果你希望匹配行且在列上广播，则必须使用算术运算方法。

In [10]:
series3 = frame['d']

In [11]:
frame

,b,d,e
Utah,0,1,2
Ohio,3,4,5
Texas,6,7,8
Oregon,9,10,11


In [12]:
series3

Utah       1
Ohio       4
Texas      7
Oregon    10
Name: d, dtype: int32

In [13]:
frame.sub(series3, axis=0)

,b,d,e
Utah,-1,0,1
Ohio,-1,0,1
Texas,-1,0,1
Oregon,-1,0,1


传入的轴号就是希望匹配的轴。在本例中，我们的目的是匹配DataFrame的行索引并进行广播。

## 函数应用和映射

NumPy的ufuncs（元素级数组方法）也可用于操作pandas对象

In [14]:
frame = DataFrame(np.random.randn(4, 3), columns=list('bde'), index=['Utah', 'Ohio', 'Texas', 'Oregon'])
frame

,b,d,e
Utah,0.376167,-0.327892,-0.577100
Ohio,0.496159,-2.033134,-0.138881
Texas,0.035876,0.342236,1.530497
Oregon,0.992021,-0.359724,1.899928


另一个常见的操作是，将函数应用到由各列或行所形成的一维数组上。DataFrame的apply方法即可实现此功能：

In [15]:
f = lambda x: x.max() - x.min()
frame.apply(f)

b    0.956146
d    2.375369
e    2.477028
dtype: float64

In [16]:
frame.apply(f, axis=1)

Utah      0.953267
Ohio      2.529293
Texas     1.494621
Oregon    2.259653
dtype: float64

许多最为常见的数组统计功能被实现成DataFrame的方法，因此无需使用apply方法。除了标量外，传递给apply的函数还可以返回由多个值组成的Series

In [17]:
def f(x):
    return Series([x.min(), x.max()], index=['min', 'max'])
frame.apply(f)

,b,d,e
min,0.035876,-2.033134,-0.577100
max,0.992021,0.342236,1.899928


此外，元素级的Python函数也是可以用的。假如你想得到frame中各个浮点值的格式化字符串，使用applymap即可

In [18]:
format = lambda x: '%.2f' % x
frame.applymap(format)

,b,d,e
Utah,0.38,-0.33,-0.58
Ohio,0.50,-2.03,-0.14
Texas,0.04,0.34,1.53
Oregon,0.99,-0.36,1.90


之所以叫做applymap，是因为Series有一个用于应用元素级函数的map方法

In [19]:
frame['e'].map(format)

Utah      -0.58
Ohio      -0.14
Texas      1.53
Oregon     1.90
Name: e, dtype: object

## 排序和排名

根据条件对数据集排序（sorting）也是一种重要的内置运算。要对行列或索引进行排序（按字典顺序），可使用sort_index方法，它将返回一个已排序的新对象

In [20]:
obj = Series(range(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [21]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

面对于DataFrame，则可以根据任意一个轴上的索引进行排序

In [22]:
frame = DataFrame(np.arange(8).reshape((2, 4)), index=['three', 'one'], columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [13]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [23]:
frame.sort_index(axis=1)

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


数据默认是按升序排序的，但也可以降序排序

In [24]:
frame.sort_index(axis=1, ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


若要按值对Series进行排序，可使用其order方法

In [26]:
obj = Series([4, 7, -3, 2])
obj

0    4
1    7
2   -3
3    2
dtype: int64

In [28]:
# 这里不再使用order()
obj.sort_values()

2   -3
3    2
0    4
1    7
dtype: int64

在排序时，任何缺失值默认都会被放到Series的末尾

In [29]:
obj = Series([4, np.nan, 7, np.nan, -3, 2])
obj

0    4.0
1    NaN
2    7.0
3    NaN
4   -3.0
5    2.0
dtype: float64

In [30]:
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

在DataFrame上，你可能希望根据一个或多个列中的值进行排序。将一个或多个列的名字传递给by选项即可

In [31]:
frame = DataFrame({'b':[4,7,-3,2], 'a':[0,1,0,1]})
frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


In [32]:
frame.sort_values(by='b')

,b,a
2,-3,0
3,2,1
0,4,0
1,7,1


要根据多个列进行排序，传入名称的列表即可：

In [33]:
frame.sort_values(by=['a','b'])

,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


排名（ranking）跟排序关系密切，且他会增设一个排名值（从1开始，一直到数组中有效数据的数量）。它跟numpy.argsort产生的间接排序索引差不多，只不过它可以根据某种规则破坏平级关系。默认情况下，rank是通过为各组分配一个平均排名的方式破坏平级关系的。

In [35]:
obj = Series([7, -5, 7, 4, 2, 0, 4])
obj

0    7
1   -5
2    7
3    4
4    2
5    0
6    4
dtype: int64

In [36]:
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

也可以根据值在源数据中出现的顺序给出排名

In [37]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

降序排列

In [38]:
obj.rank(ascending=False, method='max')

0    2.0
1    7.0
2    2.0
3    4.0
4    5.0
5    6.0
6    4.0
dtype: float64

In [41]:
frame = DataFrame({'b':[4.3,7,-3,2], 'a':[0,1,0,1], 'c':[-2, 5, 8, -2.5]})
frame

,b,a,c
0,4.3,0,-2.0
1,7.0,1,5.0
2,-3.0,0,8.0
3,2.0,1,-2.5


In [42]:
frame.rank(axis=1)

,b,a,c
0,3.0,2.0,1.0
1,3.0,1.0,2.0
2,1.0,2.0,3.0
3,3.0,2.0,1.0


## 带有重复值的轴索引

直到目前为止，我所介绍的所有范例都有着唯一的轴标签（索引值）。虽然许多pandas函数（如reindex）都要求标签唯一，但这并不是强制性的。

In [46]:
obj = Series(range(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

索引的is_unique属性可以检测索引是否唯一

In [44]:
obj.index.is_unique

False

对于带有重复值的索引，数据选取的行为将会有些不同。如果某个索引对应多个值，则返回一个Series；而对应单个值的，则返回一个标量值。

In [47]:
obj['a']

a    0
a    1
dtype: int64

In [48]:
obj['c']

4

对DataFrame的行进行索引时也是如此

In [49]:
df = DataFrame(np.random.randn(4,3), index=['a','a','b','b'])
df

,0,1,2
a,2.201038,-0.398909,-0.600095
a,-0.454424,-0.686144,1.084422
b,-0.473864,1.509534,-0.438911
b,-0.627957,-1.292513,0.072158


In [50]:
df.loc['b']

,0,1,2
b,-0.473864,1.509534,-0.438911
b,-0.627957,-1.292513,0.072158


In [51]:
df.iloc[0]

0    2.201038
1   -0.398909
2   -0.600095
Name: a, dtype: float64

# 汇总和计算描述统计

pandas对象拥有一组常用的数学和统计方法。它们大部分都属于约简和汇总统计，用于从Series中提取单个值或从DataFrame的行或列中提取一个Series，跟对应的NumPy数组方法相比，他们都是基于没有缺失数据的假设而构建的。

In [52]:
df = DataFrame([[1.4, np.nan], [7.1, -4.5],[np.nan, np.nan], [0.75, -1.3]], index=['a','b','c','d'],columns=['one','two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


调用DataFrame的sum方法将会返回一个含有列小计的Series

In [53]:
df.sum()

one    9.25
two   -5.80
dtype: float64

传入axis=1将会按行进行求和运算：

In [54]:
df.sum(axis=1)

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

NA值会自动被排除，除非整个切片（这里指的是行或列）都是NA。通过skipna选项可以禁用此功能

In [55]:
df.mean(axis=1, skipna=False)

a      NaN
b    1.300
c      NaN
d   -0.275
dtype: float64

有些方法（如idxmin和idxmax）返回的是间接统计（比如达到最小值或最大值的索引）：

In [56]:
df.idxmax()

one    b
two    d
dtype: object

另一些方法则是累计型的：

In [57]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


还有一种方法，它既不是约简型也不是累计型。describe就是一个例子，它用于一次性产生多个汇总统计：

In [58]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


对于非数值型数据，describe会产生另外一种汇总统计：

In [61]:
obj = Series(['a', 'a', 'b', 'c'] * 4)
obj

0     a
1     a
2     b
3     c
4     a
5     a
6     b
7     c
8     a
9     a
10    b
11    c
12    a
13    a
14    b
15    c
dtype: object

In [60]:
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

## 相关系数与协方差

有些汇总统计（如相关系数和协方差）是通过参数对计算出来的。下面的数据来自Yahoo!Finance的股票价格和成交量

In [63]:
all_data = {}
for ticker in ['AAPL', 'IBM', 'MSFT', 'GOOG']:
    all_data[ticker] = web.get_data_yahoo(ticker, '1/1/2000', '1/1/2010')
price = DataFrame({tic:data['Adj Close']
            for tic, data in all_data.items()})
volume = DataFrame({tic: data['Volume']
            for tic, data in all_data.items()})

接下来计算价格的百分数变化

In [64]:
returns = price.pct_change()
returns.tail()

,AAPL,IBM,MSFT,GOOG
Date,,,,
2009-12-24,0.034340,0.004385,0.002587,0.011117
2009-12-28,0.012294,0.013326,0.005484,0.007098
2009-12-29,-0.011861,-0.003477,0.007058,-0.005571
2009-12-30,0.012147,0.005461,-0.013699,0.005376
2009-12-31,-0.004300,-0.012597,-0.015504,-0.004416


Series的corr方法用于计算两个Series中重叠的、非NA的、按索引对齐的值的相关系数。cov协方差也类似

In [65]:
returns.MSFT.corr(returns.IBM)

0.49435858597485605

In [66]:
returns.MSFT.cov(returns.IBM)

0.00021582148357610984

DataFrame的corr和cov方法将以DataFrame的形式返回完整的相关系数或协方差矩阵：

DataFrame的corr和cov方法将以DataFrame的兴衰返回完整的相关系数或协方差矩阵

In [67]:
returns.corr()

,AAPL,IBM,MSFT,GOOG
AAPL,1.000000,0.412392,0.423598,0.470676
IBM,0.412392,1.000000,0.494359,0.390689
MSFT,0.423598,0.494359,1.000000,0.443586
GOOG,0.470676,0.390689,0.443586,1.000000


In [68]:
returns.cov()

,AAPL,IBM,MSFT,GOOG
AAPL,0.001030,0.000254,0.000309,0.000303
IBM,0.000254,0.000369,0.000216,0.000142
MSFT,0.000309,0.000216,0.000516,0.000205
GOOG,0.000303,0.000142,0.000205,0.000580


利用DataFrame的corrwith方法，你可以计算其列或行跟另一个Series或DataFrame之间的相关系数。传入一个Series将会返回一个相关系数值Series

In [69]:
returns.corrwith(returns.IBM)

AAPL    0.412392
IBM     1.000000
MSFT    0.494359
GOOG    0.390689
dtype: float64

传入一个DataFrame则会计算按列名配对的相关系数。这里，我计算百分比变化与成交量的相关系数：

In [70]:
returns.corrwith(volume)

AAPL   -0.057665
IBM    -0.006592
MSFT   -0.014228
GOOG    0.062647
dtype: float64

## 唯一值、值计算以及成员资格

还有一类方法可以从一维Series的值中抽取信息

In [54]:
obj = Series(['c', 'a', 'd', 'a', 'a', 'b', 'b', 'c', 'c'])

array(['c', 'a', 'd', 'b'], dtype=object)

第一个函数是unique，它可以得到Series中的唯一值数组

In [71]:
uniques = obj.unique()
uniques

array(['a', 'b', 'c'], dtype=object)

返回的唯一值是未排序的，如果需要的话，可以对结果再次进行排序（uniques.sort（））。value_counts用于计算一个Series中各值出现的频率

In [72]:
obj.value_counts()

a    8
b    4
c    4
dtype: int64

为了便于查看，结果Series是按值频率降序排列的。value_counts还是一个顶级pandas方法，可用于任何数组或序列：

In [73]:
pd.value_counts(obj.values, sort=False)

a    8
c    4
b    4
dtype: int64

最后是isin，它用于判断矢量化集合的成员资格，可用于选取Series中或DataFrame列中数据的子集：

In [74]:
mask = obj.isin(['b', 'c'])
mask

0     False
1     False
2      True
3      True
4     False
5     False
6      True
7      True
8     False
9     False
10     True
11     True
12    False
13    False
14     True
15     True
dtype: bool

In [75]:
obj[mask]

2     b
3     c
6     b
7     c
10    b
11    c
14    b
15    c
dtype: object

有时，你可能希望得到DataFrame中多个相关列的一张柱状图。

In [76]:
data = DataFrame({'Qu1':[1,3,4,3,4],
                  'Qu2':[2,3,1,2,3],
                  'Qu3':[1,5,2,4,4]})
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


将pandas.value_counts传给该DataFrame的apply函数

In [77]:
result = data.apply(pd.value_counts).fillna(0)

In [78]:
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0


# 处理缺失数据

缺失数据（missing data）在大部分数据分析应用中都很常见。pandas的设计目标之一就是让缺失数据的处理任务尽量轻松。例如，pandas对象上的所有描述统计都排除了缺失数据。pandas使用浮点NaN表示浮点和肺腑点数组中的缺失数据。它只是一个便于被检测出来的标记而已。

In [79]:
string_data = Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [80]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

Python内置的None值也会被当做NA处理

In [81]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

我不敢说pandas的NA表现形式是最优的，但它确实很简单也很可靠。由于NumPy的数据类型体系中缺乏真正的NA数据类型或位模式，所以它是我能想到的最佳解决方案（一套简单的API以及足够全面的性能特征）。随着NumPy的不断发展，这个问题今后可能会发生变化。

## 滤除缺失数据

过滤掉缺失数据的方法有很多种。纯手工操作永远都是一个办法，但dropna可能会更实用。对于一个Series，dropna返回一个仅含有非空数据和索引值的Series：

In [82]:
data = Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [83]:
data.dropna()

0    1.0
2    3.5
4    7.0
dtype: float64

当然，也可以通过布尔型索引达到这个目的：

In [84]:
data[data.notnull()]

0    1.0
2    3.5
4    7.0
dtype: float64

而对于DataFrame对象，事情就有点复杂了。你可能希望丢弃全NA或含有NA的行或列，**dropna默认丢弃任何含有缺失值的行**

In [86]:
data = DataFrame([[1,3,4],[1,NA,NA],[NA,NA,NA],[NA,4,5]])
data

,0,1,2
0,1.0,3.0,4.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,4.0,5.0


In [87]:
cleaned = data.dropna()
cleaned

,0,1,2
0,1.0,3.0,4.0


传入`how=all`丢弃行全为NA的值

In [88]:
cleaned = data.dropna(how='all')
cleaned

,0,1,2
0,1.0,3.0,4.0
1,1.0,NaN,NaN
3,NaN,4.0,5.0


要用这种方式丢弃列，只需传入axis=1即可：

In [89]:
data[4] = NA
data

,0,1,2,4
0,1.0,3.0,4.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,4.0,5.0,NaN


In [90]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,3.0,4.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,4.0,5.0


另一个滤除DataFrame行的问题涉及时间序列数据。假设你只想留下一部分观测数据，可以用thresh参数实现此目的：

In [6]:
df = DataFrame(np.random.randn(7, 3))
df

,0,1,2
0,0.334329,-0.057107,-0.003920
1,-1.359868,-2.690434,0.600756
2,0.302174,1.057809,-1.045492
3,-1.800334,0.083527,0.577165
4,-0.121608,-0.410464,1.885261
5,-1.061289,-0.697219,-1.692158
6,-2.061875,0.274473,-1.462012


In [7]:
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
df

,0,1,2
0,0.334329,NaN,NaN
1,-1.359868,NaN,NaN
2,0.302174,NaN,-1.045492
3,-1.800334,NaN,0.577165
4,-0.121608,-0.410464,1.885261
5,-1.061289,-0.697219,-1.692158
6,-2.061875,0.274473,-1.462012


In [8]:
df.dropna(thresh=3)

,0,1,2
4,-0.121608,-0.410464,1.885261
5,-1.061289,-0.697219,-1.692158
6,-2.061875,0.274473,-1.462012


### 填充缺失数据

你可能不想滤除缺失数据（有可能会丢弃跟它有关的其他数据），而是希望通过其他方式填补哪些‘空洞’。对于大多数情况而言，fillna方法是最主要的函数。

In [9]:
df.fillna(0)

,0,1,2
0,0.334329,0.000000,0.000000
1,-1.359868,0.000000,0.000000
2,0.302174,0.000000,-1.045492
3,-1.800334,0.000000,0.577165
4,-0.121608,-0.410464,1.885261
5,-1.061289,-0.697219,-1.692158
6,-2.061875,0.274473,-1.462012


若是通过一个字典调用fillna，就可以实现对不同的列填充不同的值：

In [10]:
df.fillna({1:0.5, 3:-1})

,0,1,2
0,0.334329,0.500000,NaN
1,-1.359868,0.500000,NaN
2,0.302174,0.500000,-1.045492
3,-1.800334,0.500000,0.577165
4,-0.121608,-0.410464,1.885261
5,-1.061289,-0.697219,-1.692158
6,-2.061875,0.274473,-1.462012


fillna默认会返回新对象，但也可以对现有对象进行就地修改：

In [11]:
# 总是返回被填充对象的引用
_ = df.fillna(0, inplace=True)
df

,0,1,2
0,0.334329,0.000000,0.000000
1,-1.359868,0.000000,0.000000
2,0.302174,0.000000,-1.045492
3,-1.800334,0.000000,0.577165
4,-0.121608,-0.410464,1.885261
5,-1.061289,-0.697219,-1.692158
6,-2.061875,0.274473,-1.462012


对reindex有效的那些插值方法也可以用于fillna：

In [12]:
df = DataFrame(np.random.randn(6, 3))
df

,0,1,2
0,-1.772148,1.641589,0.667862
1,1.191479,0.233900,-0.087518
2,-1.819639,-0.349422,0.534412
3,1.142199,0.738094,-0.103432
4,-0.816783,-0.093293,-0.004693
5,0.259376,-0.994624,-0.128938


In [14]:
df.iloc[2:, 1] = NA
df

,0,1,2
0,-1.772148,1.641589,0.667862
1,1.191479,0.233900,-0.087518
2,-1.819639,NaN,0.534412
3,1.142199,NaN,-0.103432
4,-0.816783,NaN,-0.004693
5,0.259376,NaN,-0.128938


In [15]:
df.iloc[4:, 2] = NA
df

,0,1,2
0,-1.772148,1.641589,0.667862
1,1.191479,0.233900,-0.087518
2,-1.819639,NaN,0.534412
3,1.142199,NaN,-0.103432
4,-0.816783,NaN,NaN
5,0.259376,NaN,NaN


In [16]:
df.fillna(method='ffill')

,0,1,2
0,-1.772148,1.641589,0.667862
1,1.191479,0.233900,-0.087518
2,-1.819639,0.233900,0.534412
3,1.142199,0.233900,-0.103432
4,-0.816783,0.233900,-0.103432
5,0.259376,0.233900,-0.103432


In [17]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,-1.772148,1.641589,0.667862
1,1.191479,0.233900,-0.087518
2,-1.819639,0.233900,0.534412
3,1.142199,0.233900,-0.103432
4,-0.816783,NaN,-0.103432
5,0.259376,NaN,-0.103432


只要稍微动动脑子，你就可以利用fillna实现许多别的功能。比如说，你可以传入Series的平均值或中位数：

In [18]:
data = Series([1, NA, 3.5, NA, 7])
data

0    1.0
1    NaN
2    3.5
3    NaN
4    7.0
dtype: float64

In [19]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

### 层次化索引

层次化索引（hierarchical indexing）是pandas的一项重要功能，它使你能在一个轴上拥有多个索引级别。抽象点说，它使你能以低纬度形式处理高纬度数据。

In [20]:
data = Series(np.random.randn(10), index=[['a','a','a','b','b','b','c','c','d','d'],[1,2,3,1,2,3,1,2,2,3]])
data

a  1    0.832415
   2   -0.901591
   3    1.137018
b  1    0.330471
   2    0.578702
   3   -1.439325
c  1    0.697333
   2   -1.411850
d  2    0.092301
   3   -0.406084
dtype: float64

这就是带有MultiIndex索引的Series的格式化输出形式。索引之间的间隔表示直接使用上面的标签：

In [21]:
data.index

MultiIndex(levels=[['a', 'b', 'c', 'd'], [1, 2, 3]],
           codes=[[0, 0, 0, 1, 1, 1, 2, 2, 3, 3], [0, 1, 2, 0, 1, 2, 0, 1, 1, 2]])

对于一个层次化索引的对象，选取数据子集的操作很简单：

In [22]:
data['b']

1    0.330471
2    0.578702
3   -1.439325
dtype: float64

In [23]:
data['b':'c']

b  1    0.330471
   2    0.578702
   3   -1.439325
c  1    0.697333
   2   -1.411850
dtype: float64

In [24]:
data.loc[['b','d']]

b  1    0.330471
   2    0.578702
   3   -1.439325
d  2    0.092301
   3   -0.406084
dtype: float64

有时甚至还可以在内层中进行选取：

In [25]:
data[:,2]

a   -0.901591
b    0.578702
c   -1.411850
d    0.092301
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色。比如说，这段数据可以通过其unstack方法被重新安排到一个DataFrame中：

In [26]:
data.unstack()

,1,2,3
a,0.832415,-0.901591,1.137018
b,0.330471,0.578702,-1.439325
c,0.697333,-1.411850,NaN
d,NaN,0.092301,-0.406084


unstack的逆运算是stack

In [27]:
data.unstack().stack()

a  1    0.832415
   2   -0.901591
   3    1.137018
b  1    0.330471
   2    0.578702
   3   -1.439325
c  1    0.697333
   2   -1.411850
d  2    0.092301
   3   -0.406084
dtype: float64

对于一个DataFrame，每条轴都可以有分层索引：

In [28]:
frame = DataFrame(np.arange(12).reshape((4,3)), index=[['a','a','b','b'],[1,2,1,2]],columns=[['Ohio','Ohio','Colorado'],['Green','Red','Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各分层都可以有名字（可以是字符串，也可以是别的Python对象）。如果指定了名称，它们就会显示在控制台输出中（不要将索引名称跟轴标签混为一谈！）

In [30]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

由于有了分部的列索引，因此可以轻松选取列分组：

In [31]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

## 重排分级顺序

有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进行排序。swaplevel接收两个级别编号或名称，并返回一个互换级别的新对象（但数据不会发生变化）。

In [33]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

而sort_values则根据单个级别中的值对数据进行排序（稳定的）。交换级别时，常常也会用到sort_values，这样最终结果就是有序的了：

In [36]:
frame.sort_values(by='key1')

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

## 根据级别汇总统计

许多对DataFrame和Series的描述和汇总统计都有一个level选项，它用于指定在某条轴上求和的级别。再以上面那个DataFrame为例，我们可以根据行货列上的级别来进行求和。

In [37]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [38]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

这其实是利用了pandas的groupby功能

## 使用DataFrame的列

人们经常想要将DataFrame的一个或多个列当做索引来用，或者可能希望将行索引变成DataFrame的列。

In [40]:
frame = DataFrame({'a':range(7), 'b':range(7,0,-1), 'c':['one','one','one','two','two','two','two'], 'd':[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


DataFrame的set_index函数会将其一个或多个列转换为行索引，并创建一个新的DataFrame：

In [41]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来：

In [42]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

reset_index的功能跟set_index刚好相反，层次化索引的级别会被转移到列里面：

In [43]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


# 其他有关pandas的话题

## 整数索引

操作由整数索引的pandas对象常常会让新手抓狂，因为他们跟内置的Python数据结构（如列表和元组）在所有语义上有些不同。例如，你可能认为下面这段代码不会产生一个错误

In [44]:
ser = Series(np.arange(3.))
ser[-1]

KeyError: -1

在这种情况下，虽然pandas会求助于整数索引，但没有哪种方法（至少我就不知道）能够既不引入任何bug又安全有效地解决该问题。这里，我们有一个含有0、1、2的索引，但是很难推断出用户想要什么（基于标签或位置的索引）：

In [45]:
ser

0    0.0
1    1.0
2    2.0
dtype: float64

相反，对于一个非整数索引，就没有这样的歧义：

In [46]:
ser2 = Series(np.arange(3), index=['a', 'b', 'c'])
ser2

a    0
b    1
c    2
dtype: int32

In [47]:
ser2[-1]

2

为了保持良好的一致性，如果你的轴索引含有索引器，那么根据整数进行数据选取的操作将总是面向标签的，这也包括iloc进行切片

In [49]:
ser.iloc[:1]

0    0.0
dtype: float64

如果你需要可靠的、不考虑索引类型的、基于位置的索引，可以使用Series的iget_value方法和DataFrame和icol方法：

## 面板数据

pandas有一个Panel数据结构，你可以将其看做一个三维版的DataFrame。pandas的大部分开发工作都集中在表格型数据的操作上，因为这些数据更常见，而且层次化索引也是的多数情况下没必要使用真正的N维数组。

In [52]:
# 报错可能和日期索引有关，但是不知道怎么修改
pdata = pd.Panel(dict((stk, web.get_data_yahoo(stk, '1/1/2009', '6/1/2012')) for stk in ['AAPL', 'GOOG', 'MSFT', 'DELL']))

KeyError: 'Date'